In [3]:
import numpy as np
import matplotlib.pyplot as plt
import skrf as rf
from skrf import Network
from skrf import plotting
# The example uses pylab - I think this further encourages sloppy usage. 
# from pylab import *

<hr>
Modified   By   Reason <br>
17-Jan-22  CBL  Original <br>
Online docs: <br>
<ul>
<li> https://scikit-rf.readthedocs.io/en/latest/tutorials/Plotting.html </li>
<li> github associated with this: https://github.com/scikit-rf/ </li>
<li> https://scikit-rf.readthedocs.io/en/latest/examples/networktheory/Transmission%20Line%20Properties%20and%20Manipulations.html </li>
</ul>
<img src="transmission_line_properties.svg" />
<hr>

In [4]:
# skrf figure styling
rf.stylely()

In [6]:
freq = rf.Frequency(250, npoints=1, unit='MHz')
Z_0 = 75  # Ohm
Z_L = 150  # Ohm
d   = 12  # m
VF  = 0.67
att = 0.02 # Np/m. Equivalent to 0.1737 dB/m

In order to get the RF parameters of the transmission line, it is necessary to derive the propagation constant of the line. The propagation constant $\gamma$ of the line is defined in scikit-rf as $\gamma = \alpha + j \beta $ where $\alpha $ is the attenuation (in Neper/m) and $\beta = {2\pi \over {\lambda}} = {\omega \over{c}}//VF =  {\omega \over{c}} \sqrt{\epsilon_r}$ <br> 
The Wavelength of the coaxial cable is: <br> 
<center> 
$\lambda = {c\over{f \sqrt{\epsilon_r}}} = {c \over{f}}VF $
</center>

In [8]:
lambd = rf.c/freq.f * VF
print('VF=',VF, ' and Wavelength: ', lambd, 'm')

VF= 0.67  and Wavelength:  [0.80344379] m


As the attenuation is already given in Np/m, the propagation constant is:

In [9]:
alpha = att  # Np/m !
beta = freq.w/rf.c/VF
gamma = alpha + 1j*beta
print('Transmission line propagation constant: gamma = ', gamma, 'rad/m')

Transmission line propagation constant: gamma =  [0.02+7.82031725j] rad/m


If the attenuation would have been given in other units, scikit-rf provides the necessary tools to convert units. 

In [10]:
# db/m
print('Attenuation dB/m:', rf.np_2_db(att))
# db per 100m
print('Line attenuation in dB/100m:', rf.np_2_db(att)*100)
# feet
print('Line attenuation in dB/100ft:', rf.np_2_db(att)*100*rf.feet_2_meter())

Attenuation dB/m: 0.1737177927613007
Line attenuation in dB/100m: 17.37177927613007
Line attenuation in dB/100ft: 5.2949183233644455


and its associated Standing Wave Ratio (SWR) is obtained from

In [11]:
rf.zl_2_swr(Z_0, Z_L)

array([2.])

After propagating by a distance, d, in the transmission line of propagation constant $\gamma$ (having travelled an electrical length $\theta = \gamma d $ )  the reflection coefficient at the line input is obtained from

In [12]:
Gamma_in = rf.zl_2_Gamma_in(Z_0, Z_L, theta=gamma*d)
print('|Gamma_in|=', abs(Gamma_in), 'phase=', 180/rf.pi*angle(Gamma_in))

|Gamma_in|= [0.20626113] phase= [46.2918563]


The input impedance $Z_{in}$ is given by

In [13]:
Z_in = rf.zl_2_zin(Z_0, Z_L, gamma * d)
print('Input impedance Z_in=', Z_in)

Input impedance Z_in= [94.79804694+29.52482706j]


Like previously, the SWR at the line input is:

In [14]:
rf.zl_2_swr(Z_0, Z_in)

array([1.51972037])

The total line loss in dB is get from

In [15]:
rf.mag_2_db10(rf.zl_2_total_loss(Z_0, Z_L, gamma*d))

array([2.40732856])